## LAB01 Detección de pishing

#### Paula Camila Gonzalez Ortega, 18398
#### Diana Ximena de León Figueroa, 18607

In [1]:
import pandas as pd
import numpy as np
import  re
import  matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse, urlencode
# import pandas_profiling as pp
# from pandas_profiling import ProfileReport
# import sklearn
# from sklearn import metrics, model_selection, tree

In [2]:
## Caracteristicas derivadas

def H_entropy (x):
    # Entropia de Shannon
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

def posicionPrimerDigito( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

def proporcionVocalesConsonantes (x):
    x = x.lower()
    patron_vocales = re.compile('([aeiou])')
    patron_consonantes = re.compile('([b-df-hj-np-tv-z])')
    vocales = re.findall(patron_vocales, x)
    consonantes = re.findall(patron_consonantes, x)
    try:
        proporcion = len(vocales) / len(consonantes)
    except: # Division por cero
        proporcion = 0  
    return proporcion

### Exploración de datos

In [3]:
df = pd.read_csv('dataset_pishing.csv')

In [4]:
df.head()

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 67 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   ip                          11430 non-null  int64  
 2   nb_www                      11430 non-null  int64  
 3   nb_com                      11430 non-null  int64  
 4   nb_dslash                   11430 non-null  int64  
 5   http_in_path                11430 non-null  int64  
 6   punycode                    11430 non-null  int64  
 7   port                        11430 non-null  int64  
 8   tld_in_path                 11430 non-null  int64  
 9   tld_in_subdomain            11430 non-null  int64  
 10  abnormal_subdomain          11430 non-null  int64  
 11  nb_subdomains               11430 non-null  int64  
 12  prefix_suffix               11430 non-null  int64  
 13  random_domain               114

##### Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como “pishing”. ¿Está balanceado el dataset?

Esta balanceado

In [6]:
df['status'].value_counts() 

legitimate    5715
phishing      5715
Name: status, dtype: int64

### Derivación de las características

In [7]:
def getDomain(url):
    return urlparse(url).netloc
    
def getProtocol(url):
    return 1 if urlparse(url).scheme == 'https' else 0
    
def getSpecialCharacters(url):
    count_characters = 0
    count_characters += url.count('.')
    count_characters += url.count('-')
    count_characters += url.count('@')
    count_characters += url.count('?')
    count_characters += url.count('&')
    count_characters += url.count('|')
    count_characters += url.count('=')
    count_characters += url.count('_')
    count_characters += url.count('~')
    count_characters += url.count('%')
    count_characters += url.count('/')
    count_characters += url.count('*')
    count_characters += url.count(':')
    count_characters += url.count(',')
    count_characters += url.count(';')
    count_characters += url.count('$')
    count_characters += url.count('%20')
    count_characters += url.count(' ')
    return count_characters

In [8]:
df['longitud_url'] = df['url'].str.len()
df['longitud_hostname'] = df['url'].apply(getDomain).str.len()
df['special_characters'] = df['url'].apply(getSpecialCharacters)
df['is_https'] = df['url'].apply(getProtocol)
df['ratio_digits_url'] = df['url'].str.count('[0-9]') / df['url'].str.len()
df['ratio_digits_domain'] = df['url'].apply(getDomain).str.count('[0-9]') / df['url'].apply(getDomain).str.len()

In [9]:
df['longitud_url']

0         37
1         77
2        126
3         18
4         55
        ... 
11425     45
11426     84
11427    105
11428     38
11429    477
Name: longitud_url, Length: 11430, dtype: int64

In [10]:
df['longitud_hostname']

0        19
1        23
2        50
3        11
4        15
         ..
11425    17
11426    18
11427    16
11428    30
11429    14
Name: longitud_hostname, Length: 11430, dtype: int64

In [11]:
df['special_characters']

0         7
1         7
2        19
3         5
4        10
         ..
11425     7
11426    16
11427    17
11428     6
11429    99
Name: special_characters, Length: 11430, dtype: int64

In [12]:
df['is_https']

0        0
1        0
2        1
3        0
4        0
        ..
11425    0
11426    0
11427    1
11428    0
11429    0
Name: is_https, Length: 11430, dtype: int64

In [13]:
df['ratio_digits_url']

0        0.000000
1        0.220779
2        0.150794
3        0.000000
4        0.000000
           ...   
11425    0.000000
11426    0.023810
11427    0.142857
11428    0.000000
11429    0.085954
Name: ratio_digits_url, Length: 11430, dtype: float64

In [14]:
df['ratio_digits_domain']

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
11425    0.000000
11426    0.000000
11427    0.000000
11428    0.000000
11429    0.785714
Name: ratio_digits_domain, Length: 11430, dtype: float64

In [15]:
df['url']

0                    http://www.crestonwood.com/router.php
1        http://shadetreetechnology.com/V4/validation/a...
2        https://support-appleld.com.secureupdate.duila...
3                                       http://rgipt.ac.in
4        http://www.iracing.com/tracks/gateway-motorspo...
                               ...                        
11425        http://www.fontspace.com/category/blackletter
11426    http://www.budgetbots.com/server.php/Server%20...
11427    https://www.facebook.com/Interactive-Televisio...
11428               http://www.mypublicdomainpictures.com/
11429    http://174.139.46.123/ap/signin?openid.pape.ma...
Name: url, Length: 11430, dtype: object

### Ejemplo

### Preprocesamiento

In [16]:
# Codificacion de variable objetivo
df['status'] = df['status'].replace(to_replace='phishing', value = 1)
df['status'] = df['status'].replace(to_replace='legitimate', value = 0)

In [17]:
df['status']

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: int64

In [18]:
df.drop(['url'], axis = 1, inplace = True)